In [ ]:
# Install required Python libraries (Google Colab usually has these pre-installed)
!pip install pandas numpy scikit-learn matplotlib seaborn --quiet


In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload hacktrain.csv and hacktest.csv


Saving hacktest.csv to hacktest.csv
Saving hacktrain.csv to hacktrain.csv


In [ ]:
import pandas as pd
import numpy as np

# Load datasets
train = pd.read_csv("hacktrain.csv")
test = pd.read_csv("hacktest.csv")

# Basic overview
print("Train shape:", train.shape)
print("Test shape:", test.shape)
print("Target classes:", train['class'].unique())
train.head()


Train shape: (8000, 30)
Test shape: (2845, 29)
Target classes: ['water' 'forest' 'impervious' 'farm' 'grass' 'orchard']


,Unnamed: 0,ID,class,20150720_N,20150602_N,20150517_N,20150501_N,20150415_N,20150330_N,20150314_N,...,20140610_N,20140525_N,20140509_N,20140423_N,20140407_N,20140322_N,20140218_N,20140202_N,20140117_N,20140101_N
0,0,1,water,637.5950,658.668,-1882.030,-1924.36,997.904,-1739.990,630.087,...,NaN,-1043.160,-1942.490,267.138,NaN,NaN,211.328,-2203.020,-1180.19,433.906
1,1,2,water,634.2400,593.705,-1625.790,-1672.32,914.198,-692.386,707.626,...,NaN,-933.934,-625.385,120.059,364.858,476.972,220.878,-2250.000,-1360.56,524.075
2,3,4,water,58.0174,-1599.160,NaN,-1052.63,NaN,-1564.630,NaN,...,-1025.88,368.622,NaN,-1227.800,304.621,NaN,369.214,-2202.120,NaN,-1343.550
3,4,5,water,72.5180,NaN,380.436,-1256.93,515.805,-1413.180,-802.942,...,-1813.95,155.624,NaN,-924.073,432.150,282.833,298.320,-2197.360,NaN,-826.727
4,7,8,water,1136.4400,NaN,NaN,1647.83,1935.800,NaN,2158.980,...,1535.00,1959.430,-279.317,-384.915,-113.406,1020.720,1660.650,-116.801,-568.05,-1357.140


In [ ]:
ndvi_cols = [col for col in train.columns if '_N' in col]
print(f"NDVI columns: {len(ndvi_cols)} found.")


NDVI columns: 27 found.


In [ ]:
# Fill missing NDVI values using column median (robust to outliers)
train[ndvi_cols] = train[ndvi_cols].fillna(train[ndvi_cols].median())
test[ndvi_cols] = test[ndvi_cols].fillna(train[ndvi_cols].median())


In [ ]:
# Add NDVI stats: mean, std, max, min
for df in [train, test]:
    df['ndvi_mean'] = df[ndvi_cols].mean(axis=1)
    df['ndvi_std'] = df[ndvi_cols].std(axis=1)
    df['ndvi_max'] = df[ndvi_cols].max(axis=1)
    df['ndvi_min'] = df[ndvi_cols].min(axis=1)

# Final feature list
feature_cols = ndvi_cols + ['ndvi_mean', 'ndvi_std', 'ndvi_max', 'ndvi_min']


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

le = LabelEncoder()
train['target'] = le.fit_transform(train['class'])

X = train[feature_cols]
y = train['target']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(test[feature_cols])


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')
model.fit(X_train, y_train)

val_preds = model.predict(X_val)
print("✅ Validation Accuracy:", accuracy_score(y_val, val_preds))


✅ Validation Accuracy: 0.915


In [ ]:
# Predict and convert labels back
test_preds = model.predict(X_test)
test_labels = le.inverse_transform(test_preds)

# Submission DataFrame
submission = pd.DataFrame({
    'ID': test['ID'],
    'class': test_labels
})

# Save and download
submission.to_csv("submission.csv", index=False)
print("✅ submission.csv created.")


✅ submission.csv created.


In [ ]:
from google.colab import files
files.download("submission.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>